In [ ]:
# GUI for user to enter the queue

In [2]:
import tkinter as tk
import time
import firebase_admin
from firebase_admin import credentials, db
from PIL import Image, ImageTk, ImageFilter

linesNum = 5

def InitFirebase():
    # Initialize Firebase Admin SDK with your JSON key file
    cred = credentials.Certificate('sq.json')
    if firebase_admin._DEFAULT_APP_NAME not in firebase_admin._apps:
        firebase_admin.initialize_app(cred, {
            'databaseURL': 'https://smartqueue-60f8d-default-rtdb.europe-west1.firebasedatabase.app'
        })

def GetLines(size):
    res = []
    refStr = "UsersData/ONdzE675Ynf9M0yZ4IsIxMjkEVv1/readings/"
    for i in range(size):
        lineRef = refStr + str(i)
        ref = db.reference(lineRef)
        data = ref.get()
        res.append(data['num_of_people'])
    return res

def GetShortestLine():
    lines = GetLines(linesNum)
    return lines.index(min(lines))+1

def DisplayArrowTwoLines(line):
    if line == 1:
        leftArrowImageLabel.place(relx=0.5, rely=0.6, anchor="center")
    else:
        rightArrowImageLabel.place(relx=0.5, rely=0.6, anchor="center")

        
def DisplayArrowEvenLines(line):
    mid1 = linesNum//2
    mid2 = mid1+1
    if line > mid2:
        rightArrowImageLabel.place(relx=0.5, rely=0.6, anchor="center")
    elif line < mid1:
        leftArrowImageLabel.place(relx=0.5, rely=0.6, anchor="center")
    else:
        upArrowImageLabel.place(relx=0.5, rely=0.6, anchor="center")
        
def DisplayArrowOddLines(line):
    mid = linesNum//2+1
    if line > mid:
        rightArrowImageLabel.place(relx=0.5, rely=0.6, anchor="center")
    elif line < mid:
        leftArrowImageLabel.place(relx=0.5, rely=0.6, anchor="center")
    else:
        upArrowImageLabel.place(relx=0.5, rely=0.6, anchor="center")
        
def DisplayArrow(line):
    if linesNum == 1:
        upArrowImageLabel.place(relx=0.5, rely=0.6, anchor="center")
    elif linesNum == 2:
        DisplayArrowTwoLines(line)
    elif linesNum%2 == 0:
        DisplayArrowEvenLines(line)
    else:
        DisplayArrowOddLines(line)

def ForgetArrows():
    rightArrowImageLabel.place_forget()
    leftArrowImageLabel.place_forget()
    upArrowImageLabel.place_forget()

# Function to switch to the new page
def switch_to_new_page():
    line = GetShortestLine()
    image_label.place_forget()
    DisplayArrow(line)
    message_label.config(text="Go To Line Number " + str(line), bg = "white")  # Update the message
#     button_label.config(state=tk.DISABLED)  # Disable the button
    button_label.place_forget()
    window.after(3000, switch_to_previous_page)  # Switch back after 3 seconds

# Function to switch back to the previous page
def switch_to_previous_page():
    image_label.place(relx=0.5, rely=0.7, anchor="center")
#     button_label.config(state=tk.NORMAL)  # Enable the button again
    button_label.place(relx=0.5, rely=0.4, anchor="center")
    message_label.config(text="")  # Clear the message
    ForgetArrows()



InitFirebase()

# Create the main window
window = tk.Tk()
window.title("Welcome to the Queue")

# Get the screen dimensions
screen_width = window.winfo_screenwidth()
screen_height = window.winfo_screenheight()

# Set the window size as the screen size
window_width = int(screen_width)
window_height = int(screen_height)

# Calculate the window position to center it on the screen
x = (screen_width - window_width) // 2
y = (screen_height - window_height) // 2

window.geometry(f"{window_width}x{window_height}+{x}+{y}")

# Create a white background
canvas = tk.Canvas(window, width=window_width, height=window_height, bg="white")
canvas.pack(fill=tk.BOTH, expand=True)  # Fill the entire window

#image
image = Image.open("people.jpg")  # Replace with the actual path to your image file

# Calculate the new image size to fit the screen
new_width = int(0.5*screen_width)  # Adjust the scaling factor as needed
new_height = int((new_width / image.width) * image.height)

# Resize the image
image = image.resize((new_width, new_height), Image.LANCZOS)
photo = ImageTk.PhotoImage(image)
image_label = tk.Label(window, image=photo, borderwidth = 0)
image_label.place(relx=0.5, rely=0.7, anchor="center")

#image
rightArrowImage = Image.open("rightArrow.jpg")  # Replace with the actual path to your image file
rightArrowPhoto = ImageTk.PhotoImage(rightArrowImage)
rightArrowImageLabel = tk.Label(window, image=rightArrowPhoto, borderwidth = 0)
#image
leftArrowImage = Image.open("leftArrow.jpg")  # Replace with the actual path to your image file
leftArrowPhoto = ImageTk.PhotoImage(leftArrowImage)
leftArrowImageLabel = tk.Label(window, image=leftArrowPhoto, borderwidth = 0)
#image
upArrowImage = Image.open("upArrow.jpg")  # Replace with the actual path to your image file
upArrowPhoto = ImageTk.PhotoImage(upArrowImage)
upArrowImageLabel = tk.Label(window, image=upArrowPhoto, borderwidth = 0)
# Create a label for the welcome message
welcome_label = tk.Label(window, text="Welcome", font=("Helvetica", int(window_width * 0.04)), bg = "white")
welcome_label.place(relx=0.5, rely=0.2, anchor="center")

# # Create a button for entering the queue
# button_width = int(window_width * 0.2)
# button_height = int(window_height * 0.2)
# enter_queue_button = tk.Button(window, text="Enter Queue", font=("Helvetica", int(window_width * 0.015)), bg="blue", command=switch_to_new_page)
# enter_queue_button.place(relx=0.5, rely=0.4, anchor="center", width=button_width, height=button_height)
button_radius = int(window_height * 0.25)

# Load your image
button_image_file = Image.open("button.png") 
button_image_file = button_image_file.resize((button_radius, button_radius), Image.LANCZOS)
button_image = ImageTk.PhotoImage(button_image_file)  # Replace with the actual path to your image file

# Create a label for the button with the image background
button_label = tk.Label(window, image=button_image, text="Enter Queue", font=("Helvetica", int(window_width * 0.015)), compound=tk.CENTER, bg="white", padx=10, pady=10)
button_label.place(relx=0.5, rely=0.4, anchor="center")

# Bind a function to the label to perform an action when clicked
button_label.bind("<Button-1>", lambda event: switch_to_new_page())


# Create a label for the message (initially empty)
message_label = tk.Label(window, text="", font=("Helvetica", int(window_width * 0.04)), bg = "white")
message_label.place(relx=0.5, rely=0.75, anchor="center")

# Start the GUI main loop
window.mainloop()
